In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import pennylane as qml
from tqdm import tqdm
import time

# Define classifiers
CLASSIFIERS = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42)#,
    #'ExtraTrees': ExtraTreesClassifier(n_estimators=100, random_state=42),
    #'KNN': KNeighborsClassifier(n_neighbors=5)
}

def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    df=data

    
    
    from sklearn.utils import resample
    
    # Check the unique counts in the label column
    print(df['class'].value_counts())
    df = df.drop_duplicates()
    df = df.dropna()
    # Separate classes
    class_counts = df['class'].value_counts()
    minority_class = class_counts.idxmin()  # Class with fewer samples
    majority_class = class_counts.idxmax()  # Class with more samples
    
    df_minority = df[df['class'] == minority_class]
    df_majority = df[df['class'] == majority_class]
    
    # Undersample both classes to 70 records each
    df_minority_undersampled = resample(df_minority, 
                                        replace=False, 
                                        n_samples=500, 
                                        random_state=42)
    
    df_majority_undersampled = resample(df_majority, 
                                        replace=False, 
                                        n_samples=500, 
                                        random_state=42)

    # Combine the undersampled data
    df_balanced = pd.concat([df_minority_undersampled, df_majority_undersampled])
    
    # Shuffle the dataset to mix the classes
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Check new class distribution
    print(df_balanced['class'].value_counts())

    df=df_balanced
    

    from sklearn.preprocessing import LabelEncoder
    
    # List of binary categorical columns
    #binary_cols = ['direction', 'sourceTCPFlagsDescription', 'destinationTCPFlagsDescription', 'class','appName', 'protocolName']
    #le = LabelEncoder()
    
    # Apply Label Encoding for binary categorical features
    #for col in binary_cols:
        #df[col] = le.fit_transform(df[col])

    from sklearn.preprocessing import MaxAbsScaler

    #scaler = MaxAbsScaler()
    #df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    
    # Check processed data
    #print(df_scaled.head())

    from sklearn.model_selection import train_test_split
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    le = LabelEncoder()
    y = le.fit_transform(y)
    
    # For amplitude embedding, we need to normalize the data to have L2 norm = 1
    # This is different from angle embedding which uses [-π, π] range
    
    return X, y

def normalize_for_amplitude_embedding(X):
    """Normalize data for amplitude embedding (L2 norm = 1)"""
    X_normalized = []
    for x in X:
        # Calculate L2 norm
        norm = np.linalg.norm(x)
        if norm > 0:
            # Normalize the vector
            x_normalized = x / norm
        else:
            x_normalized = x
        X_normalized.append(x_normalized)
    return np.array(X_normalized)

def create_quantum_embedding_circuit(n_qubits):
    # Calculate how many qubits we need for amplitude embedding
    # For n features, we need log2(n) qubits
    n_required_qubits = int(np.ceil(np.log2(n_qubits)))
    dev = qml.device("default.qubit", wires=n_required_qubits)
    
    @qml.qnode(dev)
    def quantum_circuit(inputs):
        # Amplitude embedding for normalized features
        # This will encode the data into the amplitudes of the quantum state
        qml.AmplitudeEmbedding(inputs, wires=range(n_required_qubits), normalize=True, pad_with=0.0)
        
        # Add entangling layers for quantum advantage
        for i in range(2):  # Two layers of entanglement
            for j in range(n_required_qubits-1):
                qml.CNOT(wires=[j, j+1])
            qml.RY(np.pi/4, wires=n_required_qubits-1)
            
        # Measure in computational basis
        return [qml.expval(qml.PauliZ(i)) for i in range(n_required_qubits)]
    
    return quantum_circuit

def apply_quantum_embedding(X, n_qubits):
    # For amplitude embedding, we need to pad the data to a power of 2
    n_required_qubits = int(np.ceil(np.log2(n_qubits)))
    n_amplitudes = 2**n_required_qubits
    
    # Normalize data for amplitude embedding
    X_normalized = normalize_for_amplitude_embedding(X)
    
    quantum_circuit = create_quantum_embedding_circuit(n_qubits)
    X_quantum = []
    
    for sample in tqdm(X_normalized, desc=f"Quantum embedding (qubits={n_required_qubits})"):
        # Truncate or pad the sample to match required amplitudes
        sample_padded = np.zeros(n_amplitudes)
        sample_padded[:min(len(sample), n_amplitudes)] = sample[:min(len(sample), n_amplitudes)]
        
        # Renormalize after padding
        norm = np.linalg.norm(sample_padded)
        if norm > 0:
            sample_padded = sample_padded / norm
            
        embedded_sample = quantum_circuit(sample_padded)
        X_quantum.append(embedded_sample)
    
    return np.array(X_quantum)

# Improved Quantum KNN implementation with fidelity-based distance
class QuantumKNN:
    def __init__(self, n_neighbors=5, n_qubits=None):
        self.n_neighbors = n_neighbors
        self.n_qubits = n_qubits
        self.X_train = None
        self.y_train = None
        
    def create_fidelity_circuit(self):
        # Calculate required qubits for amplitude embedding
        n_required_qubits = int(np.ceil(np.log2(self.n_qubits)))
        
        # Create a device with the required number of qubits
        dev = qml.device("default.qubit", wires=n_required_qubits)
        
        @qml.qnode(dev)
        def state_preparation(x):
            # Prepare quantum state using amplitude embedding
            qml.AmplitudeEmbedding(x, wires=range(n_required_qubits), normalize=True, pad_with=0.0)
            # Return state
            return qml.state()
        
        return state_preparation
    
    def quantum_distance(self, x1, x2):
        # Calculate quantum distance based on fidelity between states
        state_prep = self.create_fidelity_circuit()
        
        # Pad inputs to power of 2 for amplitude embedding
        n_required_qubits = int(np.ceil(np.log2(self.n_qubits)))
        n_amplitudes = 2**n_required_qubits
        
        x1_padded = np.zeros(n_amplitudes)
        x1_padded[:min(len(x1), n_amplitudes)] = x1[:min(len(x1), n_amplitudes)]
        norm1 = np.linalg.norm(x1_padded)
        if norm1 > 0:
            x1_padded = x1_padded / norm1
            
        x2_padded = np.zeros(n_amplitudes)
        x2_padded[:min(len(x2), n_amplitudes)] = x2[:min(len(x2), n_amplitudes)]
        norm2 = np.linalg.norm(x2_padded)
        if norm2 > 0:
            x2_padded = x2_padded / norm2
        
        state1 = state_prep(x1_padded)
        state2 = state_prep(x2_padded)
        
        # Calculate fidelity (overlap) between quantum states
        fidelity = np.abs(np.vdot(state1, state2))**2
        
        # Convert to distance (1 - fidelity)
        # Higher fidelity = more similar = less distance
        return 1 - fidelity
    
    def fit(self, X, y):
        if self.n_qubits is None:
            self.n_qubits = X.shape[1]
        self.X_train = X
        self.y_train = y
        return self
    
    def predict(self, X):
        if self.n_qubits is None:
            self.n_qubits = X.shape[1]
        
        y_pred = []
        
        for test_sample in tqdm(X, desc="Quantum KNN prediction"):
            distances = []
            for train_sample in self.X_train:
                # Calculate quantum distance
                dist = self.quantum_distance(test_sample, train_sample)
                distances.append(dist)
            
            # Find k nearest neighbors
            indices = np.argsort(distances)[:self.n_neighbors]
            neighbors_classes = [self.y_train[i] for i in indices]
            
            # Majority vote
            if len(neighbors_classes) > 0:
                y_pred.append(max(set(neighbors_classes), key=neighbors_classes.count))
            else:
                # Fallback if no neighbors found
                y_pred.append(0)
        
        return np.array(y_pred)

# Fix for the metrics calculation to handle class imbalance
def fixed_evaluate_model(model, X_train, X_test, y_train, y_test):
    start_time = time.time()
    model.fit(X_train, y_train)
    fit_time = time.time() - start_time
    
    start_time = time.time()
    y_pred = model.predict(X_test)
    predict_time = time.time() - start_time
    
    # Check if we have enough classes for metrics
    unique_classes = np.unique(np.concatenate([y_test, y_pred]))
    
    # Set default metrics
    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Fit Time': fit_time,
        'Predict Time': predict_time
    }
    
    # Only calculate other metrics if we have multiple classes
    if len(unique_classes) > 1:
        metrics.update({
            'Precision': precision_score(y_test, y_pred, zero_division=0),
            'Recall': recall_score(y_test, y_pred, zero_division=0),
            'F1': f1_score(y_test, y_pred, zero_division=0),
            'Kappa': cohen_kappa_score(y_test, y_pred)
        })
        
        # Only calculate ROC AUC if both true and pred have multiple classes
        if len(np.unique(y_test)) > 1 and len(np.unique(y_pred)) > 1:
            metrics['ROC AUC'] = roc_auc_score(y_test, y_pred)
        else:
            metrics['ROC AUC'] = np.nan
    else:
        metrics.update({
            'Precision': np.nan,
            'Recall': np.nan,
            'F1': np.nan,
            'ROC AUC': np.nan,
            'Kappa': np.nan
        })
    
    return metrics

def main():
    X, y = load_and_preprocess_data('Defacement_Infogain.csv')
    
    pca_dims = [15]  # [11, 15]
    results = []
    
    for pca_dim in pca_dims:
        print(f"\nProcessing PCA dimension: {pca_dim}")
        # Use stratified split to ensure class balance
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

        #import numpy as np


        #from sklearn.model_selection import train_test_split
        #X = df.iloc[:, :-1]
        #y = df.iloc[:, -1]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7,random_state=42)
        
        from sklearn.preprocessing import MaxAbsScaler
        scaler = MaxAbsScaler()
        X_train = scaler.fit_transform(X_train)
        
        X_test = scaler.transform(X_test)
        X_test = np.clip(X_test,0,1)

        #X_train = X_train.to_numpy()  # Convert to NumPy if it's a DataFrame
        #X_test = X_test.to_numpy()

       

        
        
        # Apply quantum embedding
        X_train_quantum = apply_quantum_embedding(X_train[:, :pca_dim], pca_dim)
        X_test_quantum = apply_quantum_embedding(X_test[:, :pca_dim], pca_dim)
        
        for clf_name, clf in CLASSIFIERS.items():
            # Classical version
            clf_classical = clf.__class__(**clf.get_params())
            metrics = fixed_evaluate_model(clf_classical, X_train[:, :pca_dim], X_test[:, :pca_dim], 
                                        y_train, y_test)
            metrics['Model'] = f'{clf_name}_Classical_PCA{pca_dim}'
            metrics['PCA_Dim'] = pca_dim
            results.append(metrics)
            
            # Quantum version with classical ML models
            clf_quantum = clf.__class__(**clf.get_params())
            metrics = fixed_evaluate_model(clf_quantum, X_train_quantum, X_test_quantum, 
                                        y_train, y_test)
            metrics['Model'] = f'{clf_name}_Quantum_PCA{pca_dim}'
            metrics['PCA_Dim'] = pca_dim
            results.append(metrics)
        
        # True Quantum KNN
        print("Running True Quantum KNN with quantum distance...")
        qknn = QuantumKNN(n_neighbors=5, n_qubits=pca_dim)
        
        # Use 2000 records for training instead of 100
        sample_size = min(200000, len(X_train_quantum))
        indices = np.random.choice(len(X_train_quantum), sample_size, replace=False)
        
        # Get class distribution
                # Get class distribution
        classes, counts = np.unique(y_train[indices], return_counts=True)
        print(f"Class distribution in training sample: {dict(zip(classes, counts))}")
        
        X_train_quantum_sample = X_train_quantum[indices]
        y_train_sample = y_train[indices]
        
        # Use 400 records for testing instead of 50
        test_sample_size = min(200000, len(X_test_quantum))
        test_indices = np.random.choice(len(X_test_quantum), test_sample_size, replace=False)
        X_test_quantum_sample = X_test_quantum[test_indices]
        y_test_sample = y_test[test_indices]
        
        # Print class distribution for test set
        classes, counts = np.unique(y_test_sample, return_counts=True)
        print(f"Class distribution in test sample: {dict(zip(classes, counts))}")
        
        try:
            metrics = fixed_evaluate_model(qknn, X_train_quantum_sample, X_test_quantum_sample, 
                                        y_train_sample, y_test_sample)
            metrics['Model'] = f'TrueQuantumKNN_PCA{pca_dim}'
            metrics['PCA_Dim'] = pca_dim
            # Add sample size info to metrics
            metrics['Train_Samples'] = len(X_train_quantum_sample)
            metrics['Test_Samples'] = len(X_test_quantum_sample)
            results.append(metrics)
        except Exception as e:
            print(f"Error running True Quantum KNN: {e}")
            # Add fallback metrics with error note
            metrics = {
                'Accuracy': np.nan, 'Precision': np.nan, 'Recall': np.nan,
                'F1': np.nan, 'ROC AUC': np.nan, 'Kappa': np.nan,
                'Fit Time': np.nan, 'Predict Time': np.nan,
                'Model': f'TrueQuantumKNN_PCA{pca_dim}_Error',
                'PCA_Dim': pca_dim
            }
            results.append(metrics)
    
    results_df = pd.DataFrame(results)
    print("\nResults:")
    print(results_df.to_string())
    
    # Save results with timestamp
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    #results_df.to_csv(f'quantum_ml_results_{timestamp}.csv', index=False)

if __name__ == "__main__":
    main()



class
Defacement    7930
benign        7781
Name: count, dtype: int64
class
benign        500
Defacement    500
Name: count, dtype: int64

Processing PCA dimension: 15


Quantum embedding (qubits=4): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:02<00:00, 100.66it/s]


Running True Quantum KNN with quantum distance...
Class distribution in training sample: {0: 344, 1: 356}
Class distribution in test sample: {0: 156, 1: 144}


Quantum KNN prediction: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [15:04<00:00,  3.02s/it]


Results:
   Accuracy  Fit Time  Predict Time  Precision    Recall        F1     Kappa   ROC AUC                         Model  PCA_Dim  Train_Samples  Test_Samples
0  0.970000  0.384389      0.020361   0.972028  0.965278  0.968641  0.939888  0.969818  RandomForest_Classical_PCA15       15            NaN           NaN
1  0.826667  0.391744      0.019032   0.838235  0.791667  0.814286  0.652034  0.825321    RandomForest_Quantum_PCA15       15            NaN           NaN
2  0.823333  0.000004    904.975360   0.801325  0.840278  0.820339  0.646761  0.823985          TrueQuantumKNN_PCA15       15          700.0         300.0


In [16]:
'''
Results:
   Accuracy  Fit Time  Predict Time  Precision    Recall        F1     Kappa   ROC AUC                         Model  PCA_Dim  Train_Samples  Test_Samples
0  0.970000  0.384389      0.020361   0.972028  0.965278  0.968641  0.939888  0.969818  RandomForest_Classical_PCA15       15            NaN           NaN
1  0.826667  0.391744      0.019032   0.838235  0.791667  0.814286  0.652034  0.825321    RandomForest_Quantum_PCA15       15            NaN           NaN
2  0.823333  0.000004    904.975360   0.801325  0.840278  0.820339  0.646761  0.823985          TrueQuantumKNN_PCA15       15          700.0         300.0
'''

'\nResults:\n   Accuracy  Fit Time  Predict Time  Precision    Recall        F1     Kappa   ROC AUC                         Model  PCA_Dim  Train_Samples  Test_Samples\n0  0.970000  0.384389      0.020361   0.972028  0.965278  0.968641  0.939888  0.969818  RandomForest_Classical_PCA15       15            NaN           NaN\n1  0.826667  0.391744      0.019032   0.838235  0.791667  0.814286  0.652034  0.825321    RandomForest_Quantum_PCA15       15            NaN           NaN\n2  0.823333  0.000004    904.975360   0.801325  0.840278  0.820339  0.646761  0.823985          TrueQuantumKNN_PCA15       15          700.0         300.0\n'

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import pennylane as qml
from tqdm import tqdm
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    df=data
   
    
    from sklearn.utils import resample
    
    # Check the unique counts in the label column
    print(df['class'].value_counts())
    df = df.drop_duplicates()
    df=df.dropna()
    # Separate classes
    class_counts = df['class'].value_counts()
    minority_class = class_counts.idxmin()  # Class with fewer samples
    majority_class = class_counts.idxmax()  # Class with more samples
    
    df_minority = df[df['class'] == minority_class]
    df_majority = df[df['class'] == majority_class]
    
    # Undersample both classes to 70 records each
    df_minority_undersampled = resample(df_minority, 
                                        replace=False, 
                                        n_samples=500, 
                                        random_state=42)
    
    df_majority_undersampled = resample(df_majority, 
                                        replace=False, 
                                        n_samples=500, 
                                        random_state=42)

    # Combine the undersampled data
    df_balanced = pd.concat([df_minority_undersampled, df_majority_undersampled])
    
    # Shuffle the dataset to mix the classes
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Check new class distribution
    print(df_balanced['class'].value_counts())

    df=df_balanced
    

    from sklearn.preprocessing import LabelEncoder
    
    # List of binary categorical columns
   

    from sklearn.preprocessing import MaxAbsScaler

    #scaler = MaxAbsScaler()
    #df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    
    # Check processed data
    #print(df_scaled.head())

    from sklearn.model_selection import train_test_split
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    le = LabelEncoder()
    y = le.fit_transform(y)
    
    # For amplitude embedding, we need to normalize the data to have L2 norm = 1
    # This is different from angle embedding which uses [-π, π] range
    
    return X, y


X, y = load_and_preprocess_data('Defacement_Infogain.csv')
    


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7,random_state=42)
        
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
X_train = scaler.fit_transform(X_train)
        
X_test = scaler.transform(X_test)
X_test = np.clip(X_test,0,1)



KeyError: 'URL_Type_obf_Type'

In [14]:
knn = KNeighborsClassifier(n_neighbors=5)  # You can change k here
knn.fit(X_train, y_train)

# Step 3: Predict
y_pred = knn.predict(X_test)

# Step 4: Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred)
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print("Accuracy:",accuracy)  
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.99
Accuracy: 0.99
Confusion Matrix:
 [[395   3]
 [  5 397]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       398
           1       0.99      0.99      0.99       402

    accuracy                           0.99       800
   macro avg       0.99      0.99      0.99       800
weighted avg       0.99      0.99      0.99       800



In [10]:
precision

0.9925

In [11]:
recall

0.9875621890547264

In [13]:
f1

0.9900249376558603